In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from pathlib import Path
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d
from hloc import extract_features, match_features, reconstruction, visualization, pairs_from_retrieval,pairs_from_exhaustive,retrieval
import sys
import cv2
import os
sys.path.append("/src/app")
sys.path.append("/src")

import place_utils

In [2]:
import numpy as np

def rotation_matrix_to_quaternion(R):
    trace = np.trace(R)
    if trace > 0:
        s = 2 * np.sqrt(trace + 1)
        w = 0.25 * s
        x = (R[2, 1] - R[1, 2]) / s
        y = (R[0, 2] - R[2, 0]) / s
        z = (R[1, 0] - R[0, 1]) / s
    elif (R[0, 0] > R[1, 1]) and (R[0, 0] > R[2, 2]):
        s = 2 * np.sqrt(1 + R[0, 0] - R[1, 1] - R[2, 2])
        w = (R[2, 1] - R[1, 2]) / s
        x = 0.25 * s
        y = (R[0, 1] + R[1, 0]) / s
        z = (R[0, 2] + R[2, 0]) / s
    elif R[1, 1] > R[2, 2]:
        s = 2 * np.sqrt(1 + R[1, 1] - R[0, 0] - R[2, 2])
        w = (R[0, 2] - R[2, 0]) / s
        x = (R[0, 1] + R[1, 0]) / s
        y = 0.25 * s
        z = (R[1, 2] + R[2, 1]) / s
    else:
        s = 2 * np.sqrt(1 + R[2, 2] - R[0, 0] - R[1, 1])
        w = (R[1, 0] - R[0, 1]) / s
        x = (R[0, 2] + R[2, 0]) / s
        y = (R[1, 2] + R[2, 1]) / s
        z = 0.25 * s
    return np.array([w, x, y, z])

import numpy as np

theta = np.pi

Rx180 = np.array([[1, 0, 0],
              [0, np.cos(theta), -np.sin(theta)],
              [0, np.sin(theta), np.cos(theta)]])

In [12]:
txt_file = '/src/matcher_engine/HierarchicalLocalization/PMUC_Dataset/1000Hour_placeno16/localize_result_upper_p16_v2.txt'

txt = open(txt_file, 'r').read().split("\n")[:-1]

result = ""
for row in txt:
    row = row.split(",")
    if row[1] == "ERROR": 
        result += ','.join(row) + '\n'
        continue
    name = row[0]
    tq = [float(i) for i in row[1:]]
    tx,ty,tz,qw,qx,qy,qz = tq
    tvec = np.array([tx,ty,tz])
    R = np.array([[1 - 2*qy**2 - 2*qz**2, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw],
     [2*qx*qy + 2*qz*qw, 1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
     [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx**2 - 2*qy**2]])
    cam_tvec = -1*R.T@tvec
    Rt = R.T@Rx180
    Rtq = rotation_matrix_to_quaternion(Rt)
    t,q = cam_tvec, Rtq
    result += f"{name},{t[0]},{t[1]},{t[2]},{q[0]},{q[1]},{q[2]},{q[3]}\n"
    out_file = str(txt_file).replace(".txt","") + "_transform.txt"
    open(out_file, 'w').write(result)
    print(cam_tvec, Rtq)

[ 3.15877486 30.19580625 -0.71149574] [0.47758711 0.47195365 0.53611    0.51162131]
[ 0.32086059 26.31856626 -1.651279  ] [0.67333245 0.73899759 0.01780657 0.01374385]
[ 3.56921372 29.45740992 -0.29723175] [0.57623379 0.60280415 0.39917923 0.38110066]
[ 4.06815958 28.88433936 -1.43759998] [0.57994763 0.62802844 0.39159572 0.34043182]
[ 3.30487146 30.51220778 -0.1644682 ] [0.39561331 0.38727537 0.59147644 0.58622821]
[-3.6634901  31.74426021 -0.69736754] [-0.43607367 -0.43534831  0.56591349  0.54777142]
[ 6.39146691e-03  2.71722792e+01 -2.72837286e+00] [0.65224482 0.75597641 0.04831972 0.02723187]
[ 3.47295404 31.57024014 -0.70176445] [0.42050018 0.4232092  0.59138146 0.54253253]
[-4.64248215 33.02479175 -0.90093769] [-0.34111807 -0.35872965  0.63800179  0.58983491]
[-2.46767437 31.81689336 -1.063445  ] [-0.37117376 -0.44378254  0.59375942  0.5592288 ]
[ 1.2619258  32.78323019 -0.314956  ] [0.12497206 0.14163465 0.71088079 0.67747333]
[ 2.41344529 27.73597079 -0.23533578] [0.64113018 0.

In [11]:
result

['s',
 'r',
 '1',
 '-',
 '1',
 '0',
 '1',
 '_',
 '1',
 '9',
 '6',
 '6',
 '_',
 '1',
 '6',
 '_',
 '4',
 '.',
 'j',
 'p',
 'g',
 ',',
 '3',
 '.',
 '1',
 '5',
 '8',
 '7',
 '7',
 '4',
 '8',
 '6',
 '0',
 '5',
 '3',
 '7',
 '0',
 '7',
 '4',
 '7',
 ',',
 '3',
 '0',
 '.',
 '1',
 '9',
 '5',
 '8',
 '0',
 '6',
 '2',
 '4',
 '5',
 '8',
 '5',
 '9',
 '1',
 '6',
 ',',
 '-',
 '0',
 '.',
 '7',
 '1',
 '1',
 '4',
 '9',
 '5',
 '7',
 '4',
 '4',
 '0',
 '0',
 '2',
 '6',
 '2',
 '4',
 '6',
 ',',
 '0',
 '.',
 '4',
 '7',
 '7',
 '5',
 '8',
 '7',
 '1',
 '0',
 '8',
 '7',
 '7',
 '5',
 '6',
 '8',
 '7',
 '4',
 ',',
 '0',
 '.',
 '4',
 '7',
 '1',
 '9',
 '5',
 '3',
 '6',
 '5',
 '4',
 '9',
 '2',
 '6',
 '1',
 '2',
 '6',
 ',',
 '0',
 '.',
 '5',
 '3',
 '6',
 '1',
 '1',
 '0',
 '0',
 '0',
 '2',
 '7',
 '3',
 '1',
 '2',
 '5',
 '4',
 '2',
 ',',
 '0',
 '.',
 '5',
 '1',
 '1',
 '6',
 '2',
 '1',
 '3',
 '1',
 '1',
 '2',
 '2',
 '9',
 '9',
 '4',
 '3',
 '1',
 '\n']